# Table of Contents
   
1. Introduction
2. Import and Merge Data 
3. Data Exploratory Analysis and Visualization
3. Feature Engineering
4. Machine Learning
5. Closing

# Introduction
Use the real estate datasets from Zillow competition on Kaggle website to do data exploration and analyses
The feature/predictor datasets are related with different aspect of some real estate sold in 2016 and 2017
The response datasets are the log error difference between zestimate and real sold prices
Because I started this project when the submission deadline for this stage of the Kaggle competition already passed, I will not be able to attend the competition, yet this project is still a good practice

# Import and Merge Data

In [ ]:
#data handling/modeling
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, accuracy_score
import scipy.stats as stats

# visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")  
import matplotlib as mpl

from IPython.display import display

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Import 2016 and 2017 Date and Log Error Datasets

In [ ]:
#read 2016 property file
train2016_df = pd.read_csv("../Data/train_2016_v2.csv")
train2017_df = pd.read_csv("../Data/train_2017.csv")
display(train2016_df.columns)
display(train2017_df.columns)

In [ ]:
display(train2016_df.shape)
display(train2017_df.shape)

In [ ]:
display(train2016_df.dtypes)
display(train2017_df.dtypes)

In [ ]:
display(train2016_df.head())
display(train2017_df.head())

### Update Date Time Format

In [ ]:
train2016_df["TransDate"] = pd.to_datetime(train2016_df.transactiondate)
train2016_df['YearMonth'] = train2016_df['TransDate'].map(lambda x: 100*x.year + x.month)
train2016_df["TransYear"] = train2016_df["TransDate"].dt.year
train2016_df["TransMonth"] = train2016_df["TransDate"].dt.month
train2016_df.drop('transactiondate', axis=1, inplace=True)
display(train2016_df.head(10))

In [ ]:
train2017_df["TransDate"] = pd.to_datetime(train2017_df.transactiondate)
train2017_df['YearMonth'] = train2017_df['TransDate'].map(lambda x: 100*x.year + x.month)
train2017_df["TransYear"] = train2017_df["TransDate"].dt.year
train2017_df["TransMonth"] = train2017_df["TransDate"].dt.month
train2017_df.drop('transactiondate', axis=1, inplace=True)
display(train2017_df.head(10))

### Import 2016 and 2017 Feature Datasets

In [ ]:
data2016_df = pd.read_csv("../Data/properties_2016.csv")
data2016_df.head(5)

In [ ]:
data2017_df = pd.read_csv("../Data/properties_2017.csv")
data2017_df.head(5)

In [ ]:
display(data2016_df.shape)
display(data2017_df.shape)

In [ ]:
problem_df = data2016_df.iloc[:,[22,32,34,49,55]]
problem_df.head(20)

It seems that the four columns of data that caused warning not only have mixed data types, but also have either some data that I couldn't translate or have too many NaN data, therefore I will just drop these data

In [ ]:
data2016_df.drop(data2016_df.columns[[22,32,34,49,55]], axis=1, inplace=True)

In [ ]:
data2017_df.drop(data2017_df.columns[[22,32,34,49,55]], axis=1, inplace=True)

In [ ]:
display(data2016_df.shape)
display(data2017_df.shape)

### Merge Dataframes Together

In [ ]:
full2016_df = pd.merge(train2016_df, data2016_df, how = 'inner', on = ['parcelid'])
full2017_df = pd.merge(train2017_df, data2017_df, how = 'inner', on = ['parcelid'])
display(full2016_df.shape)
display(full2017_df.shape)

In [ ]:
full_df = full2016_df.append(full2017_df)
full_df = full_df.set_index([list(range(0,167888))])
display(full_df.shape)
display(full_df.tail(5))

# Data Exploratory Analysis and Visualization

In [ ]:
plt.subplots(figsize=(20,15))
sns.heatmap(full_df.corr())
#print (full2016_df.corr())

In [ ]:
data_correlations = full_df.corr()
data_correlations

### Analyze Null Values

In [ ]:
null_No = full_df.isnull().sum()

In [ ]:
full_df.shape[0]

In [ ]:
null_ratio = null_No/full_df.shape[0]

In [ ]:
nullRatio = pd.DataFrame(null_ratio,columns=["Null Ratio"])

In [ ]:
nullRatio.sort_values("Null Ratio",ascending=False,inplace=True)

In [ ]:
nullRatio.head(10)

In [ ]:
col_name = list(nullRatio.index)

In [ ]:
y_pos = np.arange(len(col_name))

In [ ]:
fig, ax = plt.subplots(figsize=(14,18))
plt.barh(y_pos, nullRatio['Null Ratio'], align='center', color='red')
plt.yticks(y_pos, col_name);
ax.set_xlabel("Percentage of missing values")
ax.set_title("Percentage of missing values in each column")

### Predictors Comparisons

A few predictors have similar names. Plot them out and see their relationships

In [ ]:
sns.pairplot(full_df, x_vars=["roomcnt"], y_vars=['calculatedbathnbr'], size=6)

In [ ]:
sns.pairplot(full_df, x_vars=["TransYear"], y_vars=['assessmentyear'], size=6)

In [ ]:
sns.pairplot(full_df, x_vars=["finishedsquarefeet12"], y_vars=['calculatedfinishedsquarefeet'], size=6)

In [ ]:
#plot of logerror change with both quantitative and qualitative variables 
sns.pairplot(full_df, x_vars=["yearbuilt"], y_vars=["logerror"], hue='heatingorsystemtypeid', size=6, palette='husl')
sns.pairplot(full_df, x_vars=["TransMonth"], y_vars=["logerror"], hue='propertylandusetypeid', size=6, palette='husl')
sns.pairplot(full_df, x_vars=['lotsizesquarefeet'], y_vars=['logerror'], hue='buildingqualitytypeid', size=6, palette='husl')

In [ ]:
#different room count impact on log error
sns.pairplot(full_df, x_vars=["bedroomcnt","bathroomcnt","roomcnt","calculatedbathnbr"], y_vars=["logerror"], size=6)

In [ ]:
#Tax value impact on log error
sns.pairplot(full_df, x_vars=["taxvaluedollarcnt","landtaxvaluedollarcnt","taxamount"], y_vars=["logerror"], size=6)

Based on the analyses above, key features are selected and divided into three groups: numeric features, catgorical features and location related features

In [ ]:
num_names = ['TransYear','TransMonth','yearbuilt','calculatedfinishedsquarefeet','lotsizesquarefeet','calculatedbathnbr']
cat_names = ['propertylandusetypeid','buildingqualitytypeid','heatingorsystemtypeid']
loc_names = ['latitude','longitude','regionidzip']

In [ ]:
full_df.regionidzip.nunique()

In [ ]:
full_df.regionidzip.describe()

# Feature Engineering

### Fill in Null Values
null values are filled in differently according to their different types

#### Fill in numeric columns according to their median values

In [ ]:
#show the numer of missing values per column
num_col = full_df[num_names]
print("\nNumber of missing values per column")
print(num_col.isnull().sum())

In [ ]:
#find the median value for each column
median_per_column = full_df[num_names].apply(lambda x: x.median(),axis=0)
print ("Median value per column:\n",median_per_column)

In [ ]:
#fill in with median values
num_df = full_df[num_names].fillna(median_per_column,axis=0)
print("numer of missing values after filling in")
display(num_df.isnull().sum())
display(num_df.tail(10))

#### Fill in categorical columns using their most frequent values

In [ ]:
#show the numer of missing values per column
cat_col = full_df[cat_names]
print("\nNumber of missing values per column")
print(cat_col.isnull().sum())

In [ ]:
#find the most frequent number for each column
def get_most_frequent_value(my_column):
    return my_column.value_counts().index[0]

most_frequent_values_per_column = full_df[cat_names].apply(get_most_frequent_value,axis=0)
print ("Most frequent value in each column:\n",most_frequent_values_per_column)

In [ ]:
#fill in with most frequent values
cat_df = full_df[cat_names].fillna(most_frequent_values_per_column,axis=0)
cat_df = cat_df.astype(int) 
print("numer of missing values after filling in")
display(cat_df.isnull().sum())
display(cat_df.tail(10))
cat_df.shape

#### Fill in location columns with 0 to replace NaN

In [ ]:
#show the numer of missing values per column
loc_col = full_df[loc_names]
print("\nNumber of missing values per column")
print(loc_col.isnull().sum())

In [ ]:
#fill in with 0
loc_df = full_df[loc_names].fillna(0,axis=0)
print("numer of missing values after filling in")
display(loc_df.isnull().sum())
display(loc_df.head(10))

### Transform categorical features

#### Property land use type ID

In [ ]:
#First look at its distribution
LandType = cat_df.propertylandusetypeid.value_counts()
LandType.plot(kind='bar', title="Property Land Type Counts")
plt.ylabel("Counts")
plt.xlabel("Property Land Type");

form four variables for the property land type according to the above types:
1. SFR: Single Family Residential (261)
2. CON: Condominium (266)
3. DUP: Duplex (2 Units, Any Combination) (246)
4. PUD: Planned Unit Development (269)

In [ ]:
#generate these new feature columns
col = ['SFR','CON','DUP','PUD']
typ = [261,266,246,269]
for a in range(4):
    cat_df[col[a]] = cat_df.propertylandusetypeid == typ[a]
    cat_df[col[a]] = cat_df[col[a]].astype(int)
cat_df.head(10)

#### Building quality type ID

In [ ]:
QualityType = cat_df.buildingqualitytypeid.value_counts()
QualityType.plot(kind='bar', title="Quality Type Counts")
plt.ylabel("Counts")
plt.xlabel("Quality Type");

Form four variables for the quality type according to their distributions:
1. Q7: 7
2. Q4: 4
3. Q1: 1
4. Q10: 10

In [ ]:
#generate these new feature columns
col = ['Q7','Q4','Q1','Q10']
typ = [7,4,1,10]
for a in range(4):
    cat_df[col[a]] = cat_df.buildingqualitytypeid == typ[a]
    cat_df[col[a]] = cat_df[col[a]].astype(int)
cat_df.head(10)

#### Heating system type ID

In [ ]:
HeatType = cat_df.heatingorsystemtypeid.value_counts()
HeatType.plot(kind='bar', title="Heat System Type Counts")
plt.ylabel("Counts")
plt.xlabel("Heat System Type");

In [ ]:
#generate these new feature columns
col = ['Central','FW','Yes','FAir']
typ = [2,7,24,6]
for a in range(4):
    cat_df[col[a]] = cat_df.heatingorsystemtypeid == typ[a]
    cat_df[col[a]] = cat_df[col[a]].astype(int)
cat_df.tail(10)

In [ ]:
newCat_df = cat_df.iloc[:,3:]
display(newCat_df.tail(5))
newCat_df.shape

### Scale numeric features

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() #create a scaler object
scaler.fit(num_df) #fit the scaler
num_scaled = scaler.transform(num_df) #transform the data with it
newNum_df = pd.DataFrame(num_scaled,columns=num_names)
display(newNum_df.tail(5))
newNum_df.shape

Action Item: Analyze and minimize skew

#### Feature columns after engineering

In [ ]:
feat_df = pd.concat((newNum_df,newCat_df),axis=1)
display(feat_df.head(5))
feat_df.shape

In [ ]:
#name response column
loger_df = full_df['logerror']
display(loger_df.head(5))
loger_df.shape

# Machine Learning

### Use two methods to fit data individually and then use the relative more accurate one for data prediction
1. Method One: Regression -- interpretable fitting results
2. Method Two: Random Forrest -- more flexible, possibly more accurate fitting

### Fitting Data Using Linear Regression

#### Train Test Data Split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(feat_df, loger_df, test_size=0.4, random_state=1234)

#### Linear Regression Model

In [ ]:
#train on training set
linreg = LinearRegression()
linreg.fit(X_train, y_train)

#generate predictions on training set and evaluate
y_pred_train = linreg.predict(X_train)
print( "Training set RMSE:",np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

#generate predictions on test set and evaluate
y_pred_test = linreg.predict(X_test)
print( "Test set RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

#### Regression Model with Second Order Interaction Only

Some of the features seem to interact with each other (for instance, a house with large sqft is also more likely to have a large yard) therefore I would like to also include second order interaction variables into the inputs to see if that can result in a better model fitting

In [ ]:
#generate the interactions of numeric features
from sklearn.preprocessing import PolynomialFeatures
poly_fit_2 = PolynomialFeatures(degree=2,interaction_only=True)
X_train_ = poly_fit_2.fit_transform(X_train)
X_test_ = poly_fit_2.fit_transform(X_test)
X_train.shape

In [ ]:
#train on training set
linreg = LinearRegression()
linreg.fit(X_train_, y_train)

#generate predictions on training set and evaluate
y_pred_train_ = linreg.predict(X_train_)
print( "Training set RMSE:",np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_)))

#generate predictions on test set and evaluate
y_pred_test_ = linreg.predict(X_test_)
print( "Test set RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_)))

In [ ]:
#Use Cross Validation
mean_squared_errors = np.abs(cross_val_score(linreg,feat_df,loger_df,cv=50,scoring='neg_mean_squared_error'))
root_mean_squared_errors = list(map(np.sqrt,mean_squared_errors))
print ("50-fold mean RMSE: ", np.mean(root_mean_squared_errors))

### Fitting Data Using Random Forrest

In [ ]:
#Use the same set of features, yet without numeric values scaled
feat_df2 = pd.concat((num_df,newCat_df),axis=1)
X_train2,X_test2,y_train2,y_test2 = train_test_split(feat_df2, loger_df, test_size=0.4, random_state=123)

In [ ]:
rfreg = RandomForestRegressor(n_estimators=200, max_features=18, bootstrap=True, oob_score=True, random_state=123)
rfreg.fit(X_train2,y_train2)
preds = rfreg.predict(X_test2)
rmse = np.sqrt(metrics.mean_squared_error(y_test2,preds))
print("RMSE:",rmse)

In [ ]:
#Use the same train test split datasets from the regression model (with numeric features scaled too)
rfreg = RandomForestRegressor(n_estimators=200, max_features=18, bootstrap=True, oob_score=True, random_state=12)
rfreg.fit(X_train,y_train)
preds = rfreg.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test,preds))
print("RMSE:",rmse)

In [ ]:
feat_names = list(feat_df2.columns)

In [ ]:
# compute feature importances
pd.DataFrame({'feature':feat_names, 
              'importance':rfreg.feature_importances_}).sort_values(by='importance',ascending=False)

# Closing

I was not able to finish everything planned in order to build a better model.
Other things that I planned to do for this project include:
1. Add other important features into the model, such as all the region related features; try a few more other features such as basementsqft, garagetotalsqft and update features with the best selections
2. Also drop the not-so-relevant features that I've chosen by examing the P values to see if they make statistical difference; and try to use PCA
3. Try to optimize the parameters used in Random Forrest Model such as n_estimators and max_features
4. Try other methods and algorithms that may bring better fitting and predictions